In [1]:
# Based off of the original Space Invader AI tutorial by Nicholas Renotte
# URL Link: https://github.com/nicknochnack/KerasRL-OpenAI-Atari-SpaceInvadersv0/blob/main/Space%20Invaders%20Walkthrough.ipynb

In [2]:
# Import Dependencies
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [1]:
#!pip install gym[atari]
#!pip uninstall gym[atari]

Found existing installation: gym 0.19.0
Uninstalling gym-0.19.0:
  Would remove:
    /home/maxbrette/.local/lib/python3.8/site-packages/gym-0.19.0.dist-info/*
    /home/maxbrette/.local/lib/python3.8/site-packages/gym/*
  Would not remove (might be manually added):
    /home/maxbrette/.local/lib/python3.8/site-packages/gym/envs/atari/environment.py
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [4]:
#!pip install atari-py
#!pip uninstall atari-py

In [5]:
!pip install gym[accept-rom-license]

In [13]:
!pip install pyglet

     |████████████████████████████████| 1.1 MB 1.2 MB/s eta 0:00:01


In [6]:
#Test Random Environment with OpenAI Gym
import gym
import random
#import atari_py

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [7]:
#print(atari_py.list_games())

In [9]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [10]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [14]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1, 2, 3, 4, 5])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

/home/maxbrette/.local/lib/python3.8/site-packages/gym/envs/atari/environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode: 1 Score: 135.0
Episode: 2 Score: 110.0
Episode: 3 Score: 210.0
Episode: 4 Score: 95.0
Episode: 5 Score: 80.0


In [15]:
# Create a Deep Learning Model With Keras
import numpy as no
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

2022-02-08 15:27:33.247853: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-02-08 15:27:33.247880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [16]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides = (4,4), activation = 'relu', 
                            input_shape = (3, height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides = (2,2), activation = 'relu'))
    model.add(Convolution2D(64, (3,3), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [17]:
del model

NameError: name 'model' is not defined

In [23]:
model = build_model(height, width, channels, actions)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [25]:
# Build Agenth With Keras-RL
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [26]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                                  attr = 'eps', 
                                  value_max = 1., 
                                  value_min = .1, 
                                  value_test = .2, 
                                  nb_steps = 10000)
    
    memory = SequentialMemory(limit = 1000, 
                              window_length = 3)
    
    dqn = DQNAgent(model = model, 
                   memory = memory, 
                   policy = policy, 
                   enable_dueling_network = True, 
                   dueling_type = 'avg', 
                   nb_actions = actions, 
                   nb_steps_warmup = 1000)
    
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-4))

In [29]:
dqn.fit(env, nb_steps = 10000, visualize = False, verbose = 2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 1015/10000: episode: 1, duration: 56.502s, episode steps: 1015, steps per second:  18, episode reward: 255.000, mean reward:  0.251 [ 0.000, 30.000], mean action: 2.533 [0.000, 5.000],  loss: 308.787573, mean_q: 22.177942, mean_eps: 0.909325
 1519/10000: episode: 2, duration: 722.811s, episode steps: 504, steps per second:   1, episode reward: 55.000, mean reward:  0.109 [ 0.000, 20.000], mean action: 2.458 [0.000, 5.000],  loss: 2.004889, mean_q: 12.254759, mean_eps: 0.886015
 2671/10000: episode: 3, duration: 1533.417s, episode steps: 1152, steps per second:   1, episode reward: 300.000, mean reward:  0.260 [ 0.000, 30.000], mean action: 2.486 [0.000, 5.000],  loss: 1.084927, mean_q: 11.563277, mean_eps: 0.811495
 3788/10000: episode: 4, duration: 1491.607s, episode steps: 1117, steps per second:   1, episode reward: 220.000, mean reward: 

In [33]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 205.000, steps: 938
Episode 2: reward: 170.000, steps: 719
Episode 3: reward: 195.000, steps: 693
Episode 4: reward: 75.000, steps: 668
Episode 5: reward: 120.000, steps: 755
Episode 6: reward: 110.000, steps: 486
Episode 7: reward: 195.000, steps: 788
Episode 8: reward: 5.000, steps: 395
Episode 9: reward: 165.000, steps: 704
Episode 10: reward: 225.000, steps: 1207


NameError: name 'np' is not defined

In [32]:
# Reloading Agent From Memory
dqn.save_weights('SavedWeights/10k-Fast/dqn_weightss.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weightss.h5f')